# 读取、可视化原始数据，对其进行切割、滤波和保存

In [1]:
# 加载必要的包
import matplotlib
import pathlib
import mne

In [2]:
# 使用Matplotlib的'Qt5Agg'模式，保证MNE画图功能的正常使用
matplotlib.use('Qt5Agg')

## 下载并读取原始数据

In [3]:
# 修改路径，下载示例数据
# sample_data_dir = mne.datasets.sample.data_path('F:/Database/Multimodal_data/')
sample_data_dir = 'F:/Database/Multimodal_data/mne_data\MNE-sample-data/'

sample_data_dir = pathlib.Path(sample_data_dir)
sample_data_dir

WindowsPath('F:/Database/Multimodal_data/mne_data/MNE-sample-data')

In [4]:
# 加载原始数据
raw_path = sample_data_dir / 'MEG\sample\sample_audvis_raw.fif'
raw = mne.io.read_raw(raw_path)
raw

Opening raw data file F:\Database\Multimodal_data\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.


Measurement date,"December 03, 2002 19:01:10 GMT"
Experimenter,MEG
Participant,Unknown
Digitized points,146 points
Good channels,"203 Gradiometers, 102 Magnetometers, 9 Stimulus, 59 EEG, 1 EOG"
Bad channels,"MEG 2443, EEG 053"
EOG channels,EOG 061
ECG channels,Not available
Sampling frequency,600.61 Hz
Highpass,0.10 Hz
Lowpass,172.18 Hz


In [5]:
# 可视化原始数据
raw.plot()

Using matplotlib as 2D backend.


<MNEBrowseFigure size 2560x1494 with 5 Axes>

Channels marked as bad:
['MEG 2443', 'EEG 053']


## 事件提取及可视化

In [7]:
# 从'STIM'通道中提取事件
events = mne.find_events(raw)

320 events found on stim channel STI 014
Event IDs: [ 1  2  3  4  5 32]


In [8]:
# 事件ID标注
event_id = {
    'Auditory/Left': 1,
    'Auditory/Right': 2,
    'Visual/Left': 3,
    'Visual/Right': 4,
    'Smiley': 5,
    'Button': 32
}
event_id

{'Auditory/Left': 1,
 'Auditory/Right': 2,
 'Visual/Left': 3,
 'Visual/Right': 4,
 'Smiley': 5,
 'Button': 32}

In [20]:
len(events[events[:, 2] == 3]) + len(events[events[:, 2] == 4]) # evevts:320*3 找出ID为'Visual'的事件数量

144

In [21]:
# 带有事件标记的数据可视化
raw.plot(events=events, event_id=event_id)

<MNEBrowseFigure size 2560x1351 with 5 Axes>

Channels marked as bad:
['EEG 053', 'MEG 2443']


## 相关数据信息

In [22]:
raw.info

Measurement date,"December 03, 2002 19:01:10 GMT"
Experimenter,MEG
Participant,Unknown
Digitized points,146 points
Good channels,"203 Gradiometers, 102 Magnetometers, 9 Stimulus, 59 EEG, 1 EOG"
Bad channels,"EEG 053, MEG 2443"
EOG channels,EOG 061
ECG channels,Not available
Sampling frequency,600.61 Hz
Highpass,0.10 Hz
Lowpass,172.18 Hz


In [23]:
raw.info['meas_date'] 

datetime.datetime(2002, 12, 3, 19, 1, 10, 720100, tzinfo=datetime.timezone.utc)

In [24]:
raw.info['sfreq'] # 采样频率

600.614990234375

In [25]:
raw.info['bads'] # 查看坏导

['EEG 053', 'MEG 2443']

In [26]:
raw.ch_names[:10] # 查看前10个导联名称

['MEG 0113',
 'MEG 0112',
 'MEG 0111',
 'MEG 0122',
 'MEG 0123',
 'MEG 0121',
 'MEG 0132',
 'MEG 0133',
 'MEG 0131',
 'MEG 0143']

In [27]:
raw.info['chs'][0] # 查看第一个导联的相关信息

{'scanno': 1,
 'logno': 113,
 'kind': 1 (FIFFV_MEG_CH),
 'range': 0.00030517578125,
 'cal': 3.1600000394149674e-09,
 'coil_type': 3012 (FIFFV_COIL_VV_PLANAR_T1),
 'loc': array([-0.1066    ,  0.0464    , -0.0604    , -0.0127    ,  0.0057    ,
        -0.99990302, -0.186801  , -0.98240298, -0.0033    , -0.98232698,
         0.18674099,  0.013541  ]),
 'unit': 201 (FIFF_UNIT_T_M),
 'unit_mul': 0 (FIFF_UNITM_NONE),
 'ch_name': 'MEG 0113',
 'coord_frame': 1 (FIFFV_COORD_DEVICE)}

## 可视化电极排布

In [30]:
raw.plot_sensors(ch_type='eeg', show_names=True)

<Figure size 640x640 with 1 Axes>

In [31]:
raw.plot_sensors(kind='3d', ch_type='eeg', show_names=True)

<Figure size 640x640 with 1 Axes>

## 坏导标记

In [32]:
# 再将一个EEG导联标记为坏导，并查看拓扑图
raw.info['bads']

['EEG 053', 'MEG 2443']

In [33]:
raw.info['bads'] += ['EEG 051']
raw.plot_sensors(ch_type='eeg', show_names=True) # 新的坏导已用红色标注

<Figure size 640x640 with 1 Axes>

## 选择导联子集

In [36]:
# 将所有eeg通道挑出建立一个子集
raw_eeg = raw.copy().pick_types(meg=False, eeg=True, eog=True, exclude=[]) 
len(raw_eeg.ch_names)

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Removing projector <Projection | PCA-v1, active : False, n_channels : 102>
Removing projector <Projection | PCA-v2, active : False, n_channels : 102>
Removing projector <Projection | PCA-v3, active : False, n_channels : 102>


61

In [37]:
raw_eeg.info

Measurement date,"December 03, 2002 19:01:10 GMT"
Experimenter,MEG
Participant,Unknown
Digitized points,146 points
Good channels,"58 EEG, 1 EOG"
Bad channels,"EEG 053, EEG 051"
EOG channels,EOG 061
ECG channels,Not available
Sampling frequency,600.61 Hz
Highpass,0.10 Hz
Lowpass,172.18 Hz


In [38]:
raw_eeg.plot(events=events, event_id=event_id)

<MNEBrowseFigure size 2560x1351 with 4 Axes>

Channels marked as bad:
['EEG 053', 'EEG 051']


In [39]:
# 将所有meg通道挑出作为一个子集
raw_meg = raw.copy().pick_types(meg=True, eeg=False, eog=False, exclude=[])
len(raw_meg.ch_names)

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


306

In [40]:
raw_meg.info

Measurement date,"December 03, 2002 19:01:10 GMT"
Experimenter,MEG
Participant,Unknown
Digitized points,146 points
Good channels,"203 Gradiometers, 102 Magnetometers"
Bad channels,MEG 2443
EOG channels,Not available
ECG channels,Not available
Sampling frequency,600.61 Hz
Highpass,0.10 Hz
Lowpass,172.18 Hz


In [42]:
raw_meg.plot(events=events, event_id=event_id)

<MNEBrowseFigure size 2560x1351 with 5 Axes>

Channels marked as bad:
['MEG 2443']


## 裁剪和过滤数据

In [46]:
# 将eeg子集中的时间最大到100范围内的数据裁剪出来
raw_eeg_cropped = raw_eeg.copy().crop(tmax=100)
raw_eeg_cropped.times[-1]

99.99916914588277

In [47]:
# 对裁剪出的数据进行0.1-40Hz的带通滤波
raw_eeg_cropped.load_data() # 先将数据加载到内存中才能进行滤波操作
raw_eeg_cropped_filtered = raw_eeg_cropped.copy().filter(l_freq=0.1, h_freq=40) # 使用copy()保留滤波前数据

Reading 0 ... 60061  =      0.000 ...    99.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 19821 samples (33.001 s)



In [49]:
raw_eeg_cropped.plot(events=events, event_id=event_id)
raw_eeg_cropped_filtered.plot(events=events, event_id=event_id)

<MNEBrowseFigure size 2560x1351 with 4 Axes>

Channels marked as bad:
['EEG 053', 'EEG 051']
Channels marked as bad:
['EEG 053', 'EEG 051']


In [51]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2)

raw_eeg_cropped.plot_psd(ax=ax[0], show=False)
raw_eeg_cropped_filtered.plot_psd(ax=ax[1], show=False)

ax[0].set_title('PSD before filtering')
ax[1].set_title('PSD after filtering')
ax[1].set_xlabel('Frequency (Hz)')
fig.set_tight_layout(True)
plt.show()

NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 3.410 (s)
NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 3.410 (s)


C:\Users\pomelo\AppData\Local\Temp\ipykernel_35388\1456020143.py:12: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.show()
d:\Miniconda3\envs\MF\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
d:\Miniconda3\envs\MF\lib\site-packages\numpy\core\_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
d:\Miniconda3\envs\MF\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
d:\Miniconda3\envs\MF\lib\site-packages\numpy\core\_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
d:\Miniconda3\envs\MF\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
d:\Miniconda3\envs\MF\lib\site-packages\numpy\core\_methods.py:184: RuntimeWarning

## 保存数据

In [64]:
import os

# 指定保存路径和文件名
save_path = 'F:/Database/Multimodal_data/mne_data/MNE-sample-data/out_data/eeg_cropped_filt_raw.fif'

# 获取目录路径
directory = os.path.dirname(save_path)

# 如果目录不存在，则创建它
if not os.path.exists(directory):
    os.makedirs(directory)

raw_eeg_cropped_filtered.save(save_path, overwrite=True)

Writing F:\Database\Multimodal_data\mne_data\MNE-sample-data\MEG\out_data\eeg_cropped_filt_raw.fif
Closing F:\Database\Multimodal_data\mne_data\MNE-sample-data\MEG\out_data\eeg_cropped_filt_raw.fif
[done]
